In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm

from pathlib import Path
Path("../out/").mkdir(parents=True, exist_ok=True)

# Data preparation

In [ ]:
#manually created list of animal proportion in food items
fap = pd.read_excel(
    '../data/public/food_animal_proportion.xlsx', 
    sheet_name='manual_entry',
    usecols=('food_code','food_animal_proportion')
)

In [ ]:
#full list of food intakes
food_records = pd.read_csv(
    '../data/main/food_intake_records.csv'
)
food_records['timestamp'] = pd.to_datetime (
    food_records.Ruokailuaika
)
food_records.drop(columns=['Ruokailuaika'], inplace=True)

food_records['food_code'] = food_records['Tuotetunnus'].fillna(food_records['Reseptin/tuotteen tunnus'])

food_records = food_records.merge(
    fap, 
    left_on='food_code', 
    right_on='food_code', 
    how='left'
).copy() #copy to defragment

food_records['ase'] = food_records.ENERC * food_records.food_animal_proportion
len(food_records.index)

In [ ]:
#id mapping for families
families = pd.read_csv(
    '../data/main/redcap_children.csv',
    usecols=['id_family', 'id_child']
).dropna().rename(columns={'id_child': 'id_person'})

In [ ]:
#diet classification as reported in RedCap surveys, collated in another notebook
diet_class = pd.read_csv('../data/main/diet_class.csv')
len(diet_class.index)

In [ ]:
# aggregating per person and merging other person-level data
subjects_all = food_records.groupby(
    ['Tunnus']
).sum(
    numeric_only=True
).drop(
    columns=[
        'Kulutettu määrä',
        'Käyttömäärä',
        'food_animal_proportion'
    ]
).merge(
    families, 
    left_on='Tunnus', 
    right_on='id_person', 
    how='left'
).merge(
    diet_class,
    left_on = 'id_person',
    right_on = 'id_person',
    how='left'
)
subjects_all['asep'] = subjects_all.ase / subjects_all.ENERC
len(subjects_all.index)

In [ ]:
#reused in several plots
diet_classes=['vegan','vegetarian','mixed_diet']

In [ ]:
#calculating foor record time series indicators
df = food_records[['Tunnus','timestamp']].copy()
df['date'] = df.timestamp.dt.date

fr_dates = df.groupby('Tunnus').agg({'timestamp': ['min', 'max'], 'date': 'nunique'})
fr_dates.columns = ('fr_start','fr_end','fr_days')
#fr_dates['fr_duration'] = fr_dates.fr_end - fr_dates.fr_start

subjects_all = subjects_all.merge(
    fr_dates,
    left_on='id_person',
    right_index=True,
    how='left'
)
len(subjects_all.index)

In [ ]:
#reported diet, but food record too short
subjects_all[(subjects_all.diet_main.notnull()) & (subjects_all.fr_days<3)][['id_family','id_person','diet_main','fr_start','fr_end','fr_days']]

In [ ]:
# filtering subjects to those with a completed survey and at least 3 days food record
subjects = subjects_all[(subjects_all.diet_main.notnull()) & (subjects_all.fr_days>=3)].copy()
len(subjects.index)

In [ ]:
#essential micronutrients are total sums, they need to be normalised to daily intakes

#we are divinding the total intake in the food records by the number of distinct days
micro_nutrients = [
    'VITC',
    'F20D5N3',
    'F22D6N3',
    'FE',
    'FOL',
    'CA',
    'F18D2CN6',
    'F18D3N3'
]

for k in micro_nutrients:
    subjects[f'{k}_daily_mean'] = (subjects[k] / subjects.fr_days)

In [ ]:
#energy densities of macronutrients

e_densities = {
    'FAT': 37,
    'FAPU': 37,
    'FASAT': 37,
    'CHOAVL': 17,
    'PROT': 17,
    'FIBC': 8,
}

for k,v in e_densities.items():
    # add a column for the total energy from macro nutrient k
    subjects[f'e-{k}'] = subjects[k] * v
    # calculcate energy proportion of k
    subjects[f'ep-{k}'] = subjects[f'e-{k}'] / subjects.ENERC

#sum up the energy proportions as a sanity check, excluding FAT, as it's included in FAPU and FASAT
#l = [f'ep-{k}' for k in ('FAT','CHOAVL','PROT','FIBC')]
#eps['ep-sum'] = eps[l].sum(axis=1)

In [ ]:
#CHOLE as mg/MJ
subjects['CHOLE_per_e'] = subjects.CHOLE / (subjects.ENERC / 1000) #mg/MJ

In [ ]:
subjects['FIBC_per_e'] = subjects.FIBC / (subjects.ENERC / 1000) #g/MJ

In [ ]:
children = subjects # as of yet, the diet classification is only calculated for child particpants, so this is a placeholder

# Data outputs

In [ ]:
df = children[['ep-FAPU','e-FASAT','FIBC_per_e','CHOLE_per_e','asep']]

table = []
for c in df.columns[:-1]: #all but last column, i.e. asep
    pr, pp = pearsonr(df.asep,df[c])
    #print(pp)
    table.append(['asep',c,pr,pp])

df = pd.DataFrame(table,columns=['x','y','pearson_r','pearson_p'])
df['fdr_bh_0_05'] = multipletests(df['pearson_p'], alpha=0.05, method='fdr_bh')[0]
df = df.sort_values(by='pearson_p')
#df = df.round(3)
#display(df)
df.to_excel('../out/correlation_macro_intakes_children.xlsx')

In [ ]:
df = children.groupby('diet_main').asep.describe()
display(df)
df.to_excel('../out/asep_describe_by_diet.xlsx')

# Figures

In [ ]:
scale=10
sns.set_theme(style='white',font_scale=2)
sns.set_context("paper")

tricolor_palette=['#55a868','#dd8452','#4c72b0']

In [ ]:
from string import ascii_lowercase

markers=['o','s','^']

def regplot_asep(df,ax,y,y_label,set_xlabel=True):
    ax.set_xlim(-0.01, df.asep.max()+0.01)
    sns.regplot(ax=ax, x='asep', y=y, scatter=False, data=df)
    sns.scatterplot(
        ax=ax, 
        x='asep', 
        y=y, 
        hue='diet_main',
        hue_order=diet_classes,
        style='diet_main',
        markers=markers,
        data=df, 
        s=30*scale,
        palette=tricolor_palette,
        legend=False
    )
    if set_xlabel:
        ax.set_xlabel('Animal source energy proportion')
    else:
        ax.set_xlabel(None)
    ax.set_ylabel(y_label)


from matplotlib.lines import Line2D
def splats(n):
    return [
        Line2D(
            [0], 
            [0], 
            marker=markers[i],
            color='w',
            markerfacecolor=tricolor_palette[i],
            markersize=15
        )
        for i in range(0,n)
    ]


def fig_of_regplots(df,ys,cols=2):

    plt.close()
    sns.set_theme(style='white',font_scale=2)
    
    rows = int(len(ys)/cols)+(len(ys)%cols > 0)
    h = rows*scale
    if rows > 1: h+=2
    w = cols*scale+2

    fig, axs = plt.subplots(rows,cols,figsize=(w,h))
        
    if rows > 1:
        axs_flat = [ax for row in axs for ax in row]
    else:
        axs_flat = axs

    l = list(zip(ys.keys(),ys.values(),axs_flat))
    i = 0
    for t in l:
        ax = t[2]
        y = t[0]
        regplot_asep(df,ax,y,t[1],set_xlabel=True)
        i+=1

    for ax in axs_flat[len(ys):]:
        fig.delaxes(ax)
        
    from matplotlib.lines import Line2D
    
    plt.figlegend(splats(3),diet_classes,loc='right')
        
    return fig, axs

In [ ]:
# a multi-panel graph of macro intakes linear model correlation with asep
sns.set_context("paper")

ys = {
    'ep-FAT': 'Total Fat (E%)',
    'ep-FAPU': 'Polyunsaturated fatty acids (E%)',
    'ep-FASAT': 'Saturated fatty acids (E%)',
    'ep-CHOAVL': 'Carbohydrates (E%)',
    'ep-PROT': 'Protein (E%)',
    'FIBC_per_e': 'Fiber (g/MJ)',
}

fig, axs = fig_of_regplots(children,ys,cols=3)

#plt.show()
plt.savefig('../out/macro_intakes_children.png')
plt.close()

In [ ]:
# a multi-panel graph of fat metabolism intakes linear model correlation with asep
sns.set_context("paper")

ys = {
    'ep-FASAT': 'Saturated fatty acids (E%)',
    'ep-FAPU': 'Polyunsaturated fatty acids (E%)',
    'CHOLE_per_e': 'Cholesterol (mg/MJ)',
    'FIBC_per_e': 'Fiber (g/MJ)',    
}

fig, axs = fig_of_regplots(children,ys,cols=2)

#plt.show()
plt.savefig('../out/fat_fibc_intakes_children.png')
plt.close()

In [ ]:
# a multi-panel graph of micronutrient intakes linear model correlation with asep
sns.set_context("paper")

ys = {
    'F20D5N3_daily_mean': 'EPA (mg/d)',
    'F22D6N3_daily_mean': 'DHA (mg/d)',
    'F18D2CN6_daily_mean': 'Linoleic acid (mg/d)',
    'F18D3N3_daily_mean': 'Alpha-linoleic acid (mg/d)',
    'FE_daily_mean': 'Iron (mg/d)',
    'CA_daily_mean': 'Calcium (mg/d)',
    'FOL_daily_mean': 'Folate (HPLC) (μg/d)',
    'VITC_daily_mean': 'Ascorbic acid (mg/d)',
}

fig, axs = fig_of_regplots(children,ys,cols=3)

#plt.show()
plt.savefig('../out/micro_intakes_children.png')
plt.close()

In [ ]:
#histogram of asep values in buckets of 5% and coloured according to reported diet

fg = sns.displot(
    children,
    x='asep',
    binwidth=0.05,
    hue='diet_main',
    hue_order=diet_classes,
    multiple='stack',
    height=scale,
    palette=tricolor_palette
)
fg.axes[0,0].set_xlabel('Animal source energy proportion')
fg.axes[0,0].set_ylabel('Number of participants')

plt.savefig(f'../out/displot_asep_diet_class_children.png')
plt.close()